Get Raw Batting Data

    - Currently the model averages the recent team hitting performance
    - This does not account for the particular players in the starting lineup that day
    - e.g. If a key hitter is resting, injured, got traded, etc.
    - To begin to model this we first need to scrape the raw batter data (similar to how we got the pitching data)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)

import lxml
import html5lib
from urllib.request import urlopen
import time

from bs4 import BeautifulSoup
import requests

Let's look at Ken Griffey Jr's url: https://www.retrosheet.org/boxesetc/G/Pgrifk002.htm

In [2]:
def get_daily_season_links_batter(batter_id):
    letter = batter_id.upper()[0]
    url_prefix = 'https://www.retrosheet.org/boxesetc/'
    url = url_prefix+letter+'/P'+batter_id+'.htm'
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html=list(soup.children)


    body = list(html[2].children)[5]
    pre_texts = [x for x in body.find_all('pre')]
    secnum = np.where([x.get_text().strip().startswith('Batting Record') for x in pre_texts])[0][0]
    a_pre_texts = pre_texts[secnum].find_all('a')
    daily_season_links = [url_prefix+x.attrs['href'][3:] for x in a_pre_texts if x.get_text()=='Daily']
    return(daily_season_links)

In [3]:
get_daily_season_links_batter('grifk002')

['https://www.retrosheet.org/boxesetc/1989/Igrifk0020011989.htm',
 'https://www.retrosheet.org/boxesetc/1990/Igrifk0020021990.htm',
 'https://www.retrosheet.org/boxesetc/1991/Igrifk0020031991.htm',
 'https://www.retrosheet.org/boxesetc/1992/Igrifk0020041992.htm',
 'https://www.retrosheet.org/boxesetc/1993/Igrifk0020051993.htm',
 'https://www.retrosheet.org/boxesetc/1994/Igrifk0020061994.htm',
 'https://www.retrosheet.org/boxesetc/1995/Igrifk0020071995.htm',
 'https://www.retrosheet.org/boxesetc/1996/Igrifk0020081996.htm',
 'https://www.retrosheet.org/boxesetc/1997/Igrifk0020091997.htm',
 'https://www.retrosheet.org/boxesetc/1998/Igrifk0020101998.htm',
 'https://www.retrosheet.org/boxesetc/1999/Igrifk0020111999.htm',
 'https://www.retrosheet.org/boxesetc/2000/Igrifk0020122000.htm',
 'https://www.retrosheet.org/boxesetc/2001/Igrifk0020132001.htm',
 'https://www.retrosheet.org/boxesetc/2002/Igrifk0020142002.htm',
 'https://www.retrosheet.org/boxesetc/2003/Igrifk0020152003.htm',
 'https://

In [4]:
def get_season_batting_data(url):    
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    html=list(soup.children)[-1]
    body = list(html.children)[-1]
    sec_next = list(body.children)
    secnum = np.where(["Opponent" in str(x) for x in sec_next])[0][0]
    key_section = sec_next[secnum]
    working_part = list(key_section.children)
    p_header = working_part[0].strip().split()
    mod_header= ['at_vs','Opponent','League', 'GS', 'AB', 'R', 'H', '2B', '3B', 'HR',
       'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SH', 'SF', 'XI', 'ROE', 'GDP',
       'SB', 'CS', 'AVG', 'OBP', 'SLG', 'BP', 'Pos']

    date_list = []
    day_href_list = []
    for k in range(1,len(working_part),4):
        date_list.append(working_part[k].get_text().strip())
        day_href_list.append(working_part[k].attrs['href'])

    dblhead_num_list = []
    for k in range(2,len(working_part),4):
        dblhead_num_list.append(working_part[k].strip())

    game_href_list = []
    for k in range(3,len(working_part),4):
        game_href_list.append(working_part[k].attrs['href'])

    main_data_matrix = []
    for k in range(4,len(working_part),4):
        main_data_row = (working_part[k].strip().split())[:27]
        main_data_matrix.append(main_data_row)
    row_sizes = [len(x) for x in main_data_matrix]
    max_row_size = max(row_sizes)
    min_row_size = min(row_sizes)
    if (min_row_size == max_row_size) and (max_row_size==27):
        # Everything has all 27 columns
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header)
    elif (min_row_size == max_row_size) and (max_row_size==26):
        # Everything has 26 columns, will guess position is missing
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header[:26])
        out_df['Pos'] = ''
    elif (min_row_size == 26) and (max_row_size==27):
        # Guessing position is missing for some rows but not others
        main_data_matrix = [x if len(x)==27 else x+[''] for x in main_data_matrix]
        out_df = pd.DataFrame(main_data_matrix, columns = mod_header)
    else:
        print('finding rows with less than 26 or more than 27 entries - Returning None')
        return(None)
    out_df['Date'] = date_list
    out_df['dblhead_num'] = dblhead_num_list
    return(out_df)

In [5]:
get_season_batting_data('https://www.retrosheet.org/boxesetc/1997/Igrifk0020091997.htm')

,at_vs,Opponent,League,GS,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,XI,ROE,GDP,SB,CS,AVG,OBP,SLG,BP,Pos,Date,dblhead_num
0,VS,NY,A,1,4,2,2,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,.500,.500,2.000,3,cf,4- 1-1997,
1,VS,NY,A,1,4,2,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.500,.500,1.375,3,cf,4- 2-1997,
2,VS,BOS,A,1,3,2,1,0,0,1,1,2,0,1,0,0,0,0,0,0,0,0,.455,.538,1.364,3,cf,4- 4-1997,
3,VS,BOS,A,1,4,1,2,0,0,1,2,1,1,1,0,0,0,0,0,0,0,0,.467,.556,1.333,3,cf,4- 5-1997,
4,VS,BOS,A,1,3,2,1,0,0,1,2,1,1,0,1,0,0,0,0,0,0,0,.444,.565,1.333,3,cf,4- 6-1997,
5,VS,CLE,A,1,4,0,2,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,.455,.556,1.227,3,cf,4- 7-1997,
6,VS,CLE,A,1,5,3,3,1,0,1,4,0,0,1,0,0,0,0,0,0,0,0,.481,.563,1.259,3,cf,4- 8-1997,
7,VS,CLE,A,1,3,2,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,.467,.556,1.167,3,cf,4- 9-1997,
8,AT,BOS,A,1,3,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,.424,.537,1.061,3,cf,4-12-1997,
9,AT,BOS,A,1,4,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,.405,.511,.973,3,cf,4-13-1997,


In [6]:
# Get all the data for a particular batter

def get_full_batting_data(batter_id):
    link_list = get_daily_season_links_batter(batter_id)
    df_batting = pd.DataFrame()
    for url in link_list:
        df_batting = pd.concat((df_batting, get_season_batting_data(url)))
    return(df_batting)

In [7]:
df_ken = get_full_batting_data('grifk002')
df_ken.head()

,at_vs,Opponent,League,GS,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,XI,ROE,GDP,SB,CS,AVG,OBP,SLG,BP,Pos,Date,dblhead_num
0,AT,OAK,A,1,3,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,.333,.500,.667,2,cf,4- 3-1989,
1,AT,OAK,A,1,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,.143,.250,.286,2,cf,4- 5-1989,
2,AT,OAK,A,1,5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,.083,.154,.167,2,cf,4- 6-1989,
3,AT,CAL,A,1,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,.067,.125,.133,2,cf,4- 7-1989,
4,AT,CAL,A,1,4,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,.105,.150,.158,3,cf,4- 9-1989,


In [8]:
df_ken.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2671 entries, 0 to 32
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   at_vs        2671 non-null   object
 1   Opponent     2671 non-null   object
 2   League       2671 non-null   object
 3   GS           2671 non-null   object
 4   AB           2671 non-null   object
 5   R            2671 non-null   object
 6   H            2671 non-null   object
 7   2B           2671 non-null   object
 8   3B           2671 non-null   object
 9   HR           2671 non-null   object
 10  RBI          2671 non-null   object
 11  BB           2671 non-null   object
 12  IBB          2671 non-null   object
 13  SO           2671 non-null   object
 14  HBP          2671 non-null   object
 15  SH           2671 non-null   object
 16  SF           2671 non-null   object
 17  XI           2671 non-null   object
 18  ROE          2671 non-null   object
 19  GDP          2671 non-null   

In [9]:
df = pd.read_csv('df_bp7.csv', low_memory=False)
df.shape
df.head()

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_line,over_under_result,Strt_GS_h,Strt_GS_v,Strt_IP_h,Strt_IP_v,Strt_H_h,Strt_H_v,Strt_BFP_h,Strt_BFP_v,Strt_HR_h,Strt_HR_v,Strt_R_h,Strt_R_v,Strt_ER_h,Strt_ER_v,Strt_BB_h,Strt_BB_v,Strt_IB_h,Strt_IB_v,Strt_SO_h,Strt_SO_v,Strt_SH_h,Strt_SH_v,Strt_SF_h,Strt_SF_v,Strt_WP_h,Strt_WP_v,Strt_HBP_h,Strt_HBP_v,Strt_BK_h,Strt_BK_v,Strt_2B_h,Strt_2B_v,Strt_3B_h,Strt_3B_v,Strt_IP_real_h,Strt_IP_real_v,Strt_rollsum_IP_real_10_h,Strt_rollsum_IP_real_10_v,Strt_rollsum_H_10_h,Strt_rollsum_H_10_v,Strt_rollsum_BFP_10_h,Strt_rollsum_BFP_10_v,Strt_rollsum_HR_10_h,Strt_rollsum_HR_10_v,Strt_rollsum_R_10_h,Strt_rollsum_R_10_v,Strt_rollsum_ER_10_h,Strt_rollsum_ER_10_v,Strt_rollsum_BB_10_h,Strt_rollsum_BB_10_v,Strt_rollsum_IB_10_h,Strt_rollsum_IB_10_v,Strt_rollsum_SO_10_h,Strt_rollsum_SO_10_v,Strt_rollsum_SH_10_h,Strt_rollsum_SH_10_v,Strt_rollsum_SF_10_h,Strt_rollsum_SF_10_v,Strt_rollsum_WP_10_h,Strt_rollsum_WP_10_v,Strt_rollsum_HBP_10_h,Strt_rollsum_HBP_10_v,Strt_rollsum_BK_10_h,Strt_rollsum_BK_10_v,Strt_rollsum_2B_10_h,Strt_rollsum_2B_10_v,Strt_rollsum_3B_10_h,Strt_rollsum_3B_10_v,Strt_rollsum_IP_real_35_h,Strt_rollsum_IP_real_35_v,Strt_rollsum_H_35_h,Strt_rollsum_H_35_v,Strt_rollsum_BFP_35_h,Strt_rollsum_BFP_35_v,Strt_rollsum_HR_35_h,Strt_rollsum_HR_35_v,Strt_rollsum_R_35_h,Strt_rollsum_R_35_v,Strt_rollsum_ER_35_h,Strt_rollsum_ER_35_v,Strt_rollsum_BB_35_h,Strt_rollsum_BB_35_v,Strt_rollsum_IB_35_h,Strt_rollsum_IB_35_v,Strt_rollsum_SO_35_h,Strt_rollsum_SO_35_v,Strt_rollsum_SH_35_h,Strt_rollsum_SH_35_v,Strt_rollsum_SF_35_h,Strt_rollsum_SF_35_v,Strt_rollsum_WP_35_h,Strt_rollsum_WP_35_v,Strt_rollsum_HBP_35_h,Strt_rollsum_HBP_35_v,Strt_rollsum_BK_35_h,Strt_rollsum_BK_35_v,Strt_rollsum_2B_35_h,Strt_rollsum_2B_35_v,Strt_rollsum_3B_35_h,Strt_rollsum_3B_35_v,Strt_rollsum_IP_real_75_h,Strt_rollsum_IP_real_75_v,Strt_rollsum_H_75_h,Strt_rollsum_H_75_v,Strt_rollsum_BFP_75_h,Strt_rollsum_BFP_75_v,Strt_rollsum_HR_75_h,Strt_rollsum_HR_75_v,Strt_rol

In [10]:
batter_ids = np.array([])
for num in range(1,10):
    for suffix in ['_h','_v']:
        # Check whether this should be '_id' or '_name'
        colname = 'batter'+str(num)+'_name'+suffix
        batter_ids = np.concatenate((batter_ids, pd.unique(df[colname])))
batter_ids = pd.unique(batter_ids)

In [11]:
len(batter_ids), batter_ids[:10]

(7474,
 array(['nelsd101', 'tolab101', 'willw103', 'fordt102', 'kellp101',
        'harpt101', 'clarh101', 'unsed101', 'willm102', 'bowal001'],
       dtype=object))

In [ ]:
# Given to you in /batter_data 
# *** run only if you don't mind waiting ***
'''
for i in range(len(batter_ids)): # for everyone, do range(len(batter_ids))
    p_id = batter_ids[i]
    print(i, p_id)
    df_temp = get_full_batting_data(p_id)
    # put path here to save to a specific folder
    fname_out = '/Users/antiprotons/Desktop/DA/batter_data/batting_data_'+p_id+'.csv'
    df_temp.to_csv(fname_out, index=False)
'''